In [1]:
import sys
sys.path.insert(0, '/home/anish/ASC_ML')
from ASC_ML import dataframe_extractor as de

In [2]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Concatenate, Flatten, Input, Conv2D
from tensorflow.keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

import time

2022-02-19 02:09:23.854782: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
directory = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/weather-history-dataset/weatherHistory.csv"
dataset_list = de.DataframeExtractor_csv(directory, label_names = ["Apparent Temperature (C)"]).df_list

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/weather-history-dataset/weatherHistory.csv


In [4]:
dataset_list[0].head()

,Formatted Date,Summary,Precip Type,Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [5]:
dataset_list[1].head()

,Apparent Temperature (C)
0,7.388889
1,7.227778
2,9.377778
3,5.944444
4,6.977778


In [6]:
train_x = dataset_list[0]
train_y = dataset_list[1]

In [7]:
train_x["Loud Cover"].value_counts(dropna = False).compute()

0.0    96453
Name: Loud Cover, dtype: int64

In [8]:
train_x["Precip Type"].value_counts(dropna = False).compute()

rain    85224
snow    10712
NaN       517
Name: Precip Type, dtype: int64

In [9]:
def func(row):   
    if row['Precip Type'] == 'rain':
        return 1  
    elif row['Precip Type'] == 'snow':
        return 2
    else:
        return 0

train_x['PrecipNo'] = train_x.apply(func, axis=1)

/home/anish/anaconda3/envs/tfgpu/lib/python3.8/site-packages/dask/dataframe/core.py:5077: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'int64'))

  warnings.warn(meta_warning(meta))


In [10]:
train_x["PrecipNo"].value_counts(dropna = False).compute()

1    85224
2    10712
0      517
Name: PrecipNo, dtype: int64

In [11]:
train_x = train_x.drop(["Summary", "Precip Type", "Formatted Date", "Daily Summary", "Loud Cover"], axis=1).compute()

In [12]:
train_x['PrecipNo'] = train_x['PrecipNo'].map({0: 'No Precip', 1: 'Rain', 2: 'Snow'})
train_x = dd.get_dummies(train_x, columns=['PrecipNo'], prefix='', prefix_sep='')

In [13]:
train_x.head()

,Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Pressure (millibars),No Precip,Rain,Snow
0,9.472222,0.89,14.1197,251.0,15.8263,1015.13,0,1,0
1,9.355556,0.86,14.2646,259.0,15.8263,1015.63,0,1,0
2,9.377778,0.89,3.9284,204.0,14.9569,1015.94,0,1,0
3,8.288889,0.83,14.1036,269.0,15.8263,1016.41,0,1,0
4,8.755556,0.83,11.0446,259.0,15.8263,1016.51,0,1,0


In [14]:
input_dim = train_x.shape[1]
print(input_dim)

9


In [15]:
tf.keras.backend.clear_session()

In [16]:
def baseline_model():
    # create model
    adam_optimizer = Adam(lr = 1e-3)
    model = Sequential()
    model.add(Dense(9, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=adam_optimizer, 
                  metrics = [tf.keras.metrics.MeanAbsoluteError(), "acc"])
    return model

In [17]:
X = np.array(train_x.values)
Y = np.array(train_y.values)

In [19]:
start = time.time()

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=30, batch_size=64, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5)
results = cross_val_score(pipeline, X, Y, cv=kfold)

end = time.time()
print ("Time elapsed:", end - start)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

2022-02-18 16:55:54.227304: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-18 16:55:54.228016: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-18 16:55:54.273485: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-18 16:55:54.273856: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.695GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 312.97GiB/s
2022-02-18 16:55:54.273884: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-18 16:55:54.275017: I tensorflow/stream_executor/platform/de

Time elapsed: 145.84883975982666
Baseline: -0.87 (1.11) MSE


In [20]:
def model2():
    # create model
    adam_optimizer = Adam(lr = 1e-3)
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer=adam_optimizer, 
                  metrics = [tf.keras.metrics.MeanAbsoluteError(), "acc"])
    return model

In [22]:
start = time.time()

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model2, epochs=30, batch_size=64, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5)
results = cross_val_score(pipeline, X, Y, cv=kfold)

end = time.time()
print ("Time elapsed:", end - start)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Time elapsed: 141.44713973999023
Baseline: -0.18 (0.08) MSE


In [23]:
def model3():
    # create model
    adam_optimizer = Adam(lr = 1e-3)
    model = Sequential()
    model.add(Dense(9, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer=adam_optimizer, 
                  metrics = [tf.keras.metrics.MeanAbsoluteError(), "acc"])
    return model

In [24]:
start = time.time()

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model3, epochs=30, batch_size=64, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5)
results = cross_val_score(pipeline, X, Y, cv=kfold)

end = time.time()
print ("Time elapsed:", end - start)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Time elapsed: 148.88164520263672
Baseline: -0.18 (0.04) MSE


In [25]:
def model4():
    # create model
    adam_optimizer = Adam(lr = 1e-3)
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer=adam_optimizer, 
                  metrics = [tf.keras.metrics.MeanAbsoluteError(), "acc"])
    return model

In [26]:
start = time.time()

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model4, epochs=30, batch_size=64, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5)
results = cross_val_score(pipeline, X, Y, cv=kfold)

end = time.time()
print ("Time elapsed:", end - start)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Time elapsed: 148.58093857765198
Baseline: -0.17 (0.02) MSE


In [29]:
def model5():
    # create model
    adam_optimizer = Adam(lr = 1e-3)
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer=adam_optimizer, 
                  metrics = [tf.keras.metrics.MeanAbsoluteError(), "acc"])
    return model

In [30]:
start = time.time()

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model5, epochs=30, batch_size=64, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5)
results = cross_val_score(pipeline, X, Y, cv=kfold)

end = time.time()
print ("Time elapsed:", end - start)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Time elapsed: 158.92162919044495
Baseline: -0.15 (0.03) MSE


In [31]:
def model6():
    # create model
    adam_optimizer = Adam(lr = 1e-3)
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_absolute_error', optimizer=adam_optimizer, 
                  metrics = [tf.keras.metrics.MeanAbsoluteError(), "acc"])
    return model

In [33]:
start = time.time()

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=model6, epochs=30, batch_size=64, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=5)
results = cross_val_score(pipeline, X, Y, cv=kfold)

end = time.time()
print ("Time elapsed:", end - start)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Time elapsed: 157.42704486846924
Baseline: -0.12 (0.03) MSE


In [67]:
indices = round(X.shape[0]*0.9)

X_train = X[:indices,:]
Y_train = Y[:indices,:]
X_test = X[:(input_dim-indices),:]
Y_test = Y[:(input_dim-indices),:]

In [57]:
start = time.time()
pipeline.fit(X_train,Y_train)
prediction = pipeline.predict(X_test)
end = time.time()
print("Time elapsed:", end - start)

Time elapsed: 35.33163905143738


In [59]:
prediction = pipeline.predict(X_train)
mean_absolute_error(Y_train, prediction)

0.09937471481529861

In [65]:
prediction = pipeline.predict(X_test)
mean_absolute_error(Y_test, prediction)

0.10064993430578757

In [66]:
for i in range(10):
    print(Y_test[i], prediction[i])

[7.38888889] 7.2842026
[7.22777778] 7.124077
[9.37777778] 9.298915
[5.94444444] 5.8622494
[6.97777778] 6.955845
[7.11111111] 7.022196
[5.52222222] 5.4859056
[6.52777778] 6.451088
[10.82222222] 10.825314
[13.77222222] 13.749904


In [ ]:
import ASCML
asc = ASCML(directory = "Dataset Directory"/url = "URL")
asc.visualize()
model = asc.get_model(label_names = "......")